In [1]:
# import random as rnd
# import secrets as scr

from datetime import datetime
from random import SystemRandom

rnd = SystemRandom()


class Lotteria:
    def __init__(self,
                 max_num=90,
                 max_ex=90,
                 length=6,
                 extra=1):
        self.max_numbers = max_num
        self.max_extra = max_ex
        self.length = length
        self.extra = extra

    @staticmethod
    def pallottoliere(num):
        while True:
            yield rnd.randint(1, num)

    @staticmethod
    def choice(length, num):
        def extraction():
            rnd.shuffle(numbers := list(range(1, num+1)))
            single = rnd.choice(numbers)
            numbers.remove(single)

            return single

        return [extraction() for _ in range(length)]

    @staticmethod
    def sample(length, num):
        # numbers = list(range(1, num+1))
        [*numbers] = range(1, num+1)

        return rnd.sample(numbers, k=length)

    def int(self, length, num):
        combo = set()
        while len(combo) < length:
            combo.add(next(self.pallottoliere(num)))

        return combo

    @staticmethod
    def many(picker, length, num):
        samples = [picker(length, num) for _ in range(num)]
        rnd.shuffle(samples)

        return rnd.choice(samples)

    def __call__(self, backend='sample', many=True):
        self.backend = backend
        picker = eval(backend, {},
                      {'choice': self.choice, 'int': self.int, 'sample': self.sample})

        if not many:
            combo = picker(self.length, self.max_numbers)

            if self.extra:
                super = picker(self.extra, self.max_extra)

        else:
            combo = self.many(picker,
                              self.length,
                              self.max_numbers)

            if self.extra:
                super = self.many(picker,
                                  self.extra,
                                  self.max_extra)

        now = datetime.now().strftime("%d/%m/%Y %H:%M")

        print(f'Estrazione del: {now}', '\nnumbers combo:', *sorted(combo))
        if self.extra:
            print('Superstar:', *sorted(super))  # type: ignore

In [2]:
eurojack = Lotteria(max_num=50, max_ex=12, length=5, extra=2)

In [3]:
for b in ['choice', 'int', 'sample']:
    print('###', b, '###')
    eurojack(backend=b, many=False)

print('\n################# many ####################\n')

for b in ['choice', 'int', 'sample']:
    print('###', b, '###')
    eurojack(backend=b, many=True)

### choice ###
Estrazione del: 02/06/2023 15:57 
numbers combo: 12 28 29 34 45
Superstar: 10 11
### int ###
Estrazione del: 02/06/2023 15:57 
numbers combo: 29 41 43 47 49
Superstar: 1 9
### sample ###
Estrazione del: 02/06/2023 15:57 
numbers combo: 17 20 34 44 48
Superstar: 5 9

################# many ####################

### choice ###
Estrazione del: 02/06/2023 15:57 
numbers combo: 10 14 33 37 50
Superstar: 8 10
### int ###
Estrazione del: 02/06/2023 15:57 
numbers combo: 17 18 38 42 47
Superstar: 3 10
### sample ###
Estrazione del: 02/06/2023 15:57 
numbers combo: 11 12 22 24 45
Superstar: 9 11


In [4]:
superenalotto = Lotteria()


In [5]:
superenalotto()

Estrazione del: 02/06/2023 15:57 
numbers combo: 2 3 16 38 42 90
Superstar: 77


In [6]:
superenalotto(backend='choice')

Estrazione del: 02/06/2023 15:57 
numbers combo: 50 57 70 72 78 84
Superstar: 69


In [7]:
superenalotto(backend='int', many=False)

Estrazione del: 02/06/2023 15:57 
numbers combo: 3 11 17 21 51 77
Superstar: 3
